# Importar librerías

In [1]:
%reset
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import matplotlib.gridspec as gridspec
from sklearn import datasets
import pandas as pd
from sklearn import preprocessing
%matplotlib inline

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Dataset que se va a utilizar para el entrenamiento

In [ ]:
datos = pd.read_csv('52900_alerts.csv', sep=';', header=0, index_col = 0)
datos = datos.fillna(" ")
datos['TYPE'] = datos['TYPE'].str.lower()
datos = datos.iloc[:20000]
datos = datos.drop(columns=['NUMBER', 'DATE_CASE','REF','SUBJET','COUNT_DESTIN','COUNT_CONCERN'])
datos =  datos.apply(lambda x: pd.factorize(x)[0])

datos["CLASSIF"] =datos["CLASSIF"] / datos["CLASSIF"].max()
datos["NOT_COUNTRY"] = datos["NOT_COUNTRY"] / datos["NOT_COUNTRY"].max()
datos["PROD_CAT"] = datos["PROD_CAT"] / datos["PROD_CAT"].max()
datos["TYPE"] = datos["TYPE"] / datos["TYPE"].max()
datos["RISK_DECISION"] = datos["RISK_DECISION"] / datos["RISK_DECISION"].max()
datos["ACTION_TAKEN"] = datos["ACTION_TAKEN"] / datos["ACTION_TAKEN"].max()
datos["DISTRIBUTION_STAT"] = datos["DISTRIBUTION_STAT"] / datos["DISTRIBUTION_STAT"].max()
datos["PRODUCT"] = datos["PRODUCT"] / datos["PRODUCT"].max()
datos["HAZARDS"] = datos["HAZARDS"] / datos["HAZARDS"].max()
datos["HAZARDS_CAT"] = datos["HAZARDS_CAT"] / datos["HAZARDS_CAT"].max()
datos["COUNT_ORIGEN"] = datos["COUNT_ORIGEN"] / datos["COUNT_ORIGEN"].max()

datos_entrenar = datos.sample(frac=0.25)
datos_prueba = datos_entrenar.sample(frac=0.01)
datos_entrenar = datos_entrenar.drop(datos_prueba.index)

datos_entrenar = datos_entrenar.transpose()

datos_entrenar = np.array(datos_entrenar)
datos_prueba  = np.array(datos_prueba)

# SOM Setup
## Variables definidas

In [3]:
lado_mapa = 20
periodo = 20000
learning_rate = 0.1
numero_pixeles = 11

In [4]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
num_entradas = numero_pixeles
num_datos = len(datos_entrenar[0])
# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa / 2
matriz_pesos = np.random.random((lado_mapa, lado_mapa, num_entradas))

## Funciones para entrenar/clasificar

In [5]:
# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):
    
    #Define los arrays exp y bmu para usarlos mas tarde
    exp = np.zeros((m))
    bmu = np.zeros((m))
    
    for i in range (0, len(m_pesos)):
        for j in range(0, len(m_pesos)):
        
            #Guarda en una variable los pesos de la neurona en la posicion del bucle
            neurona_pesos = m_pesos[i][j]
                
            #clacula la resta y el exponente de cada peso entre la neurona y el dato
            for k in range(0, m):
                exp[k] = (neurona_pesos[k]-patron_entrada[k])**2
            
            #Suma los resultados para obtener la distancia euclidea
            dist_euc = sum(exp)
            
            #Si estamos en la primera posicion de la matriz nos coge esa neurona como la menor
            if i == 0 and j == 0:
                bmu_idx = [i, j]
                min_dist = dist_euc
            
            #Si la neurona en la que estamos tiene menor distancia que la que ya teniamos cogemos esta
            else:
                if dist_euc < min_dist:
                    bmu_idx = [i, j]
                    min_dist = dist_euc
     
    #Guardamos los pesos de la neurona ganadora en una sola variable
    for i in range (0, m):
        bmu[i] = m_pesos[bmu_idx[0]][bmu_idx[1]][i]
    
    #Devuelve los resultados
    return (bmu, bmu_idx)

In [6]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    learning_rate = lr_inicial + (((0 - lr_inicial) / n_iteraciones) * i)
    return learning_rate

In [7]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
    lvecindario = int(1 + vecindario_inicial * (1 - (i/n_iteraciones)))
    return lvecindario

In [8]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU / (2* (vecindario_actual**2)))

## Funciones para dibujar la salida de la red

In [9]:
def dibujar_digito(ejemplo):
    plt.figure(1, figsize=(1,1))
    plt.imshow(ejemplo.reshape(8,8), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()

## SOM Entrenamiento

In [10]:
# Entrena la red con el dataset de entrenamiento
for i in range (0, periodo):
    
    print("Iteracion: ", i)
    
    #Calcular numero random
    num_random = np.random.randint(0, num_datos)
    
    datos_entrada = np.zeros((num_entradas))
    
    #Calcular neurona ganadora
    for z in range(0, num_entradas):
        datos_entrada[z] = datos_entrenar[z][num_random] 
    
  
    
    neurona_ganadora = calcular_bmu(datos_entrada, matriz_pesos, num_entradas)

    ganadora_pesos = np.array(neurona_ganadora[0])
    ganadora_idx = neurona_ganadora[1]
    
    #Variacion learning rate
    eta = variacion_learning_rate(learning_rate, i, periodo)
    
    #Variacion vecindario
    var_vecindario = variacion_vecindario(vecindario, i, periodo)
    
    #Mover pesos ganadora
    matriz_pesos[ganadora_idx[0], ganadora_idx[1]] = ganadora_pesos + eta * (np.array(datos_entrada) - ganadora_pesos)
    
    #Mover vecindario
    for x in range(0, len(matriz_pesos)):
        for y in range(0, len(matriz_pesos)):
            
            #Movemos la neurona ganadora hacia el dato
            dist_neuronas = (ganadora_idx[0]-x)**2 + (ganadora_idx[1]-y)**2
            
            #Recorremos todas las neuronas y si estan dentro del vecindario las mueve hacia el dato
            if (dist_neuronas < var_vecindario):
                amortiguacion = decay(dist_neuronas, var_vecindario)
                matriz_pesos[x][y] = matriz_pesos[x][y] + (eta * amortiguacion) * (ganadora_pesos - matriz_pesos[x][y])
        


Iteracion:  0
Iteracion:  1
Iteracion:  2
Iteracion:  3
Iteracion:  4
Iteracion:  5
Iteracion:  6
Iteracion:  7
Iteracion:  8
Iteracion:  9
Iteracion:  10
Iteracion:  11
Iteracion:  12
Iteracion:  13
Iteracion:  14
Iteracion:  15
Iteracion:  16
Iteracion:  17
Iteracion:  18
Iteracion:  19
Iteracion:  20
Iteracion:  21
Iteracion:  22
Iteracion:  23
Iteracion:  24
Iteracion:  25
Iteracion:  26
Iteracion:  27
Iteracion:  28
Iteracion:  29
Iteracion:  30
Iteracion:  31
Iteracion:  32
Iteracion:  33
Iteracion:  34
Iteracion:  35
Iteracion:  36
Iteracion:  37
Iteracion:  38
Iteracion:  39
Iteracion:  40
Iteracion:  41
Iteracion:  42
Iteracion:  43
Iteracion:  44
Iteracion:  45
Iteracion:  46
Iteracion:  47
Iteracion:  48
Iteracion:  49
Iteracion:  50
Iteracion:  51
Iteracion:  52
Iteracion:  53
Iteracion:  54
Iteracion:  55
Iteracion:  56
Iteracion:  57
Iteracion:  58
Iteracion:  59
Iteracion:  60
Iteracion:  61
Iteracion:  62
Iteracion:  63
Iteracion:  64
Iteracion:  65
Iteracion:  66
Itera

Iteracion:  671
Iteracion:  672
Iteracion:  673
Iteracion:  674
Iteracion:  675
Iteracion:  676
Iteracion:  677
Iteracion:  678
Iteracion:  679
Iteracion:  680
Iteracion:  681
Iteracion:  682
Iteracion:  683
Iteracion:  684
Iteracion:  685
Iteracion:  686
Iteracion:  687
Iteracion:  688
Iteracion:  689
Iteracion:  690
Iteracion:  691
Iteracion:  692
Iteracion:  693
Iteracion:  694
Iteracion:  695
Iteracion:  696
Iteracion:  697
Iteracion:  698
Iteracion:  699
Iteracion:  700
Iteracion:  701
Iteracion:  702
Iteracion:  703
Iteracion:  704
Iteracion:  705
Iteracion:  706
Iteracion:  707
Iteracion:  708
Iteracion:  709
Iteracion:  710
Iteracion:  711
Iteracion:  712
Iteracion:  713
Iteracion:  714
Iteracion:  715
Iteracion:  716
Iteracion:  717
Iteracion:  718
Iteracion:  719
Iteracion:  720
Iteracion:  721
Iteracion:  722
Iteracion:  723
Iteracion:  724
Iteracion:  725
Iteracion:  726
Iteracion:  727
Iteracion:  728
Iteracion:  729
Iteracion:  730
Iteracion:  731
Iteracion:  732
Iteracio

Iteracion:  1180
Iteracion:  1181
Iteracion:  1182
Iteracion:  1183
Iteracion:  1184
Iteracion:  1185
Iteracion:  1186
Iteracion:  1187
Iteracion:  1188
Iteracion:  1189
Iteracion:  1190
Iteracion:  1191
Iteracion:  1192
Iteracion:  1193
Iteracion:  1194
Iteracion:  1195
Iteracion:  1196
Iteracion:  1197
Iteracion:  1198
Iteracion:  1199
Iteracion:  1200
Iteracion:  1201
Iteracion:  1202
Iteracion:  1203
Iteracion:  1204
Iteracion:  1205
Iteracion:  1206
Iteracion:  1207
Iteracion:  1208
Iteracion:  1209
Iteracion:  1210
Iteracion:  1211
Iteracion:  1212
Iteracion:  1213
Iteracion:  1214
Iteracion:  1215
Iteracion:  1216
Iteracion:  1217
Iteracion:  1218
Iteracion:  1219
Iteracion:  1220
Iteracion:  1221
Iteracion:  1222
Iteracion:  1223
Iteracion:  1224
Iteracion:  1225
Iteracion:  1226
Iteracion:  1227
Iteracion:  1228
Iteracion:  1229
Iteracion:  1230
Iteracion:  1231
Iteracion:  1232
Iteracion:  1233
Iteracion:  1234
Iteracion:  1235
Iteracion:  1236
Iteracion:  1237
Iteracion:  12

Iteracion:  1691
Iteracion:  1692
Iteracion:  1693
Iteracion:  1694
Iteracion:  1695
Iteracion:  1696
Iteracion:  1697
Iteracion:  1698
Iteracion:  1699
Iteracion:  1700
Iteracion:  1701
Iteracion:  1702
Iteracion:  1703
Iteracion:  1704
Iteracion:  1705
Iteracion:  1706
Iteracion:  1707
Iteracion:  1708
Iteracion:  1709
Iteracion:  1710
Iteracion:  1711
Iteracion:  1712
Iteracion:  1713
Iteracion:  1714
Iteracion:  1715
Iteracion:  1716
Iteracion:  1717
Iteracion:  1718
Iteracion:  1719
Iteracion:  1720
Iteracion:  1721
Iteracion:  1722
Iteracion:  1723
Iteracion:  1724
Iteracion:  1725
Iteracion:  1726
Iteracion:  1727
Iteracion:  1728
Iteracion:  1729
Iteracion:  1730
Iteracion:  1731
Iteracion:  1732
Iteracion:  1733
Iteracion:  1734
Iteracion:  1735
Iteracion:  1736
Iteracion:  1737
Iteracion:  1738
Iteracion:  1739
Iteracion:  1740
Iteracion:  1741
Iteracion:  1742
Iteracion:  1743
Iteracion:  1744
Iteracion:  1745
Iteracion:  1746
Iteracion:  1747
Iteracion:  1748
Iteracion:  17

Iteracion:  2211
Iteracion:  2212
Iteracion:  2213
Iteracion:  2214
Iteracion:  2215
Iteracion:  2216
Iteracion:  2217
Iteracion:  2218
Iteracion:  2219
Iteracion:  2220
Iteracion:  2221
Iteracion:  2222
Iteracion:  2223
Iteracion:  2224
Iteracion:  2225
Iteracion:  2226
Iteracion:  2227
Iteracion:  2228
Iteracion:  2229
Iteracion:  2230
Iteracion:  2231
Iteracion:  2232
Iteracion:  2233
Iteracion:  2234
Iteracion:  2235
Iteracion:  2236
Iteracion:  2237
Iteracion:  2238
Iteracion:  2239
Iteracion:  2240
Iteracion:  2241
Iteracion:  2242
Iteracion:  2243
Iteracion:  2244
Iteracion:  2245
Iteracion:  2246
Iteracion:  2247
Iteracion:  2248
Iteracion:  2249
Iteracion:  2250
Iteracion:  2251
Iteracion:  2252
Iteracion:  2253
Iteracion:  2254
Iteracion:  2255
Iteracion:  2256
Iteracion:  2257
Iteracion:  2258
Iteracion:  2259
Iteracion:  2260
Iteracion:  2261
Iteracion:  2262
Iteracion:  2263
Iteracion:  2264
Iteracion:  2265
Iteracion:  2266
Iteracion:  2267
Iteracion:  2268
Iteracion:  22

...

Iteracion:  17943
Iteracion:  17944
Iteracion:  17945
Iteracion:  17946
Iteracion:  17947
Iteracion:  17948
Iteracion:  17949
Iteracion:  17950
Iteracion:  17951
Iteracion:  17952
Iteracion:  17953
Iteracion:  17954
Iteracion:  17955
Iteracion:  17956
Iteracion:  17957
Iteracion:  17958
Iteracion:  17959
Iteracion:  17960
Iteracion:  17961
Iteracion:  17962
Iteracion:  17963
Iteracion:  17964
Iteracion:  17965
Iteracion:  17966
Iteracion:  17967
Iteracion:  17968
Iteracion:  17969
Iteracion:  17970
Iteracion:  17971
Iteracion:  17972
Iteracion:  17973
Iteracion:  17974
Iteracion:  17975
Iteracion:  17976
Iteracion:  17977
Iteracion:  17978
Iteracion:  17979
Iteracion:  17980
Iteracion:  17981
Iteracion:  17982
Iteracion:  17983
Iteracion:  17984
Iteracion:  17985
Iteracion:  17986
Iteracion:  17987
Iteracion:  17988
Iteracion:  17989
Iteracion:  17990
Iteracion:  17991
Iteracion:  17992
Iteracion:  17993
Iteracion:  17994
Iteracion:  17995
Iteracion:  17996
Iteracion:  17997
Iteracion:

Iteracion:  18443
Iteracion:  18444
Iteracion:  18445
Iteracion:  18446
Iteracion:  18447
Iteracion:  18448
Iteracion:  18449
Iteracion:  18450
Iteracion:  18451
Iteracion:  18452
Iteracion:  18453
Iteracion:  18454
Iteracion:  18455
Iteracion:  18456
Iteracion:  18457
Iteracion:  18458
Iteracion:  18459
Iteracion:  18460
Iteracion:  18461
Iteracion:  18462
Iteracion:  18463
Iteracion:  18464
Iteracion:  18465
Iteracion:  18466
Iteracion:  18467
Iteracion:  18468
Iteracion:  18469
Iteracion:  18470
Iteracion:  18471
Iteracion:  18472
Iteracion:  18473
Iteracion:  18474
Iteracion:  18475
Iteracion:  18476
Iteracion:  18477
Iteracion:  18478
Iteracion:  18479
Iteracion:  18480
Iteracion:  18481
Iteracion:  18482
Iteracion:  18483
Iteracion:  18484
Iteracion:  18485
Iteracion:  18486
Iteracion:  18487
Iteracion:  18488
Iteracion:  18489
Iteracion:  18490
Iteracion:  18491
Iteracion:  18492
Iteracion:  18493
Iteracion:  18494
Iteracion:  18495
Iteracion:  18496
Iteracion:  18497
Iteracion:

Iteracion:  18942
Iteracion:  18943
Iteracion:  18944
Iteracion:  18945
Iteracion:  18946
Iteracion:  18947
Iteracion:  18948
Iteracion:  18949
Iteracion:  18950
Iteracion:  18951
Iteracion:  18952
Iteracion:  18953
Iteracion:  18954
Iteracion:  18955
Iteracion:  18956
Iteracion:  18957
Iteracion:  18958
Iteracion:  18959
Iteracion:  18960
Iteracion:  18961
Iteracion:  18962
Iteracion:  18963
Iteracion:  18964
Iteracion:  18965
Iteracion:  18966
Iteracion:  18967
Iteracion:  18968
Iteracion:  18969
Iteracion:  18970
Iteracion:  18971
Iteracion:  18972
Iteracion:  18973
Iteracion:  18974
Iteracion:  18975
Iteracion:  18976
Iteracion:  18977
Iteracion:  18978
Iteracion:  18979
Iteracion:  18980
Iteracion:  18981
Iteracion:  18982
Iteracion:  18983
Iteracion:  18984
Iteracion:  18985
Iteracion:  18986
Iteracion:  18987
Iteracion:  18988
Iteracion:  18989
Iteracion:  18990
Iteracion:  18991
Iteracion:  18992
Iteracion:  18993
Iteracion:  18994
Iteracion:  18995
Iteracion:  18996
Iteracion:

Iteracion:  19442
Iteracion:  19443
Iteracion:  19444
Iteracion:  19445
Iteracion:  19446
Iteracion:  19447
Iteracion:  19448
Iteracion:  19449
Iteracion:  19450
Iteracion:  19451
Iteracion:  19452
Iteracion:  19453
Iteracion:  19454
Iteracion:  19455
Iteracion:  19456
Iteracion:  19457
Iteracion:  19458
Iteracion:  19459
Iteracion:  19460
Iteracion:  19461
Iteracion:  19462
Iteracion:  19463
Iteracion:  19464
Iteracion:  19465
Iteracion:  19466
Iteracion:  19467
Iteracion:  19468
Iteracion:  19469
Iteracion:  19470
Iteracion:  19471
Iteracion:  19472
Iteracion:  19473
Iteracion:  19474
Iteracion:  19475
Iteracion:  19476
Iteracion:  19477
Iteracion:  19478
Iteracion:  19479
Iteracion:  19480
Iteracion:  19481
Iteracion:  19482
Iteracion:  19483
Iteracion:  19484
Iteracion:  19485
Iteracion:  19486
Iteracion:  19487
Iteracion:  19488
Iteracion:  19489
Iteracion:  19490
Iteracion:  19491
Iteracion:  19492
Iteracion:  19493
Iteracion:  19494
Iteracion:  19495
Iteracion:  19496
Iteracion:

Iteracion:  19941
Iteracion:  19942
Iteracion:  19943
Iteracion:  19944
Iteracion:  19945
Iteracion:  19946
Iteracion:  19947
Iteracion:  19948
Iteracion:  19949
Iteracion:  19950
Iteracion:  19951
Iteracion:  19952
Iteracion:  19953
Iteracion:  19954
Iteracion:  19955
Iteracion:  19956
Iteracion:  19957
Iteracion:  19958
Iteracion:  19959
Iteracion:  19960
Iteracion:  19961
Iteracion:  19962
Iteracion:  19963
Iteracion:  19964
Iteracion:  19965
Iteracion:  19966
Iteracion:  19967
Iteracion:  19968
Iteracion:  19969
Iteracion:  19970
Iteracion:  19971
Iteracion:  19972
Iteracion:  19973
Iteracion:  19974
Iteracion:  19975
Iteracion:  19976
Iteracion:  19977
Iteracion:  19978
Iteracion:  19979
Iteracion:  19980
Iteracion:  19981
Iteracion:  19982
Iteracion:  19983
Iteracion:  19984
Iteracion:  19985
Iteracion:  19986
Iteracion:  19987
Iteracion:  19988
Iteracion:  19989
Iteracion:  19990
Iteracion:  19991
Iteracion:  19992
Iteracion:  19993
Iteracion:  19994
Iteracion:  19995
Iteracion:

## SOM Clasificación

In [11]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada

#Definir matrices a 0
matriz_clasificacion = np.zeros((lado_mapa, lado_mapa, num_entradas))
matriz_activaciones = np.zeros((lado_mapa, lado_mapa))
matriz_distancias = np.zeros((lado_mapa, lado_mapa))

exp = np.zeros((num_entradas))

for i in range(0, lado_mapa):
    for j in range(0, lado_mapa):
    
        #Calcular color random
        num_random = np.random.randint(0, num_datos)

        datos_entrada = np.zeros((num_entradas))
    
        #Calcular neurona ganadora
        for z in range(0, num_entradas):
            datos_entrada[z] = datos_entrenar[z][num_random]
        

        neurona_ganadora = calcular_bmu(datos_entrada, matriz_pesos, num_entradas)

        ganadora_pesos = np.array(neurona_ganadora[0])
        ganadora_idx = neurona_ganadora[1]
        
        #Añadir uno a la matriz de activaciones en la posicion de la neurona ganadora
        matriz_activaciones[ganadora_idx[0]][ganadora_idx[1]] += 1
        
        #Calcular la distancia euclidea entre el color y la neurona ganadora
        for k in range(0, num_entradas):
                exp[k] = (ganadora_pesos[k]-datos_entrada[k])**2
        
        dist_euc = sum(exp)
        
        #Añadir la distancia en la matriz de distancias en la posicion de la neurona ganadora
        matriz_distancias[ganadora_idx[0]][ganadora_idx[1]] += dist_euc
        #Añadir el color a la matriz clasificacion (se pisa si encuentra una nueva)
        matriz_clasificacion[ganadora_idx[0]][ganadora_idx[1]] = datos_entrada
        
contador_clases = 0 #inicializar el contador de clases       
        
for i in range(0, lado_mapa):
    for j in range(0, lado_mapa):
        
        if matriz_activaciones[i][j] > 0:
            
            #Aumentar el contador de clases y hacer la media de las distancias de cada clase
            contador_clases += 1;
            matriz_distancias[i][j] = matriz_distancias[i][j] / matriz_activaciones[i][j]

#Calcular la media total del mapa de distancias
suma_distancias = sum(sum(matriz_distancias))
media_distancias = suma_distancias / contador_clases

#pintar_mapa(matriz_clasificacion)
print("\nMatriz de activaciones: \n", matriz_activaciones)
print(matriz_activaciones.sum())
print("\nMatriz distancias: \n", matriz_distancias)
print("\nMedia: ", media_distancias)
print("\nNumero de grupos: ", contador_clases)


Matriz de activaciones: 
 [[ 7.  8.  1.  6.  5.  1.  0.  2.  1.  0.  2.  0.  1.  4.  0.  1.  2.  2.
   2.  1.]
 [ 1.  0.  0.  1.  4.  0.  0.  1.  3.  1.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  1.]
 [ 0.  1.  4.  1.  0.  0.  2.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  1.]
 [ 2.  0.  1.  0.  4.  0.  2.  0.  0.  1.  1.  0.  0.  3.  1.  1.  0.  2.
   1.  0.]
 [ 6.  0.  2.  1.  0.  0.  0.  1.  1.  0.  0.  1.  2.  0.  0.  0.  3.  0.
   0.  3.]
 [ 1.  5.  1.  3.  1.  0.  0.  0.  0.  0.  0.  1.  2.  0.  0.  0.  0.  1.
   0.  1.]
 [ 6.  0.  2.  3.  2.  0.  1.  1.  0.  4.  1.  0.  1.  3.  0.  1.  1.  0.
   0.  0.]
 [ 1.  0.  0.  0.  0.  0.  2.  0.  4.  0.  0.  0.  1.  0.  0.  1.  2. 11.
   0.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  2.
   0.  1.]
 [ 3.  2.  0.  1.  1.  1.  2.  0.  4.  2.  0.  1.  0.  2.  2.  6.  0.  0.
   0.  1.]
 [ 0.  3.  5.  0.  0.  0.  3.  0.  0.  1.  3.  0.  0.  0.  0.  0.  2.  0.
   0.  3.]
 [ 4.  2.  0.  0.  2.  3.  0.  0.  1. 

## SOM Prueba

In [12]:
# Clasifica nuevos patrones

exp = np.zeros((num_entradas))

#Datos dados en el enunciado


#datos_prueba = datos_prueba.transpose()

datos_prueba  = list(datos_prueba)


for i in range(0, len(datos_prueba)):

    #Reshape d los datos dados en el enunciado
    datos_entrada = datos_prueba[i]
    
    #Calcular neurona ganadora
    neurona_ganadora = calcular_bmu(datos_entrada, matriz_pesos, num_entradas)

    ganadora_pesos = np.array(neurona_ganadora[0])
    ganadora_idx = neurona_ganadora[1]

    #Añadir uno a la matriz de activaciones en la posicion de la neurona ganadora
    matriz_activaciones[ganadora_idx[0]][ganadora_idx[1]] += 1

    #Calcular la distancia euclidea entre el numero y la neurona ganadora
    for k in range(0, num_entradas):
            exp[k] = (ganadora_pesos[k]-datos_entrada[k])**2

    dist_euc = sum(exp)

    #Añadir la distancia en la matriz de distancias en la posicion de la neurona ganadora
    matriz_distancias[ganadora_idx[0]][ganadora_idx[1]] += dist_euc
    #Añadir el numero a la matriz clasificacion (se pisa si encuentra una nueva)
    matriz_clasificacion[ganadora_idx[0]][ganadora_idx[1]] = datos_entrada
        
contador_clases = 0 #inicializar el contador de clases       
        
for i in range(0, lado_mapa):
    for j in range(0, lado_mapa):
        
        if matriz_activaciones[i][j] > 0:
            
            #Aumentar el contador de clases y hacer la media de las distancias de cada clase
            contador_clases += 1;
            matriz_distancias[i][j] = matriz_distancias[i][j] / matriz_activaciones[i][j]

#Calcular la media total del mapa de distancias
suma_distancias = sum(sum(matriz_distancias))
media_distancias = suma_distancias / contador_clases

#pintar_mapa(matriz_clasificacion)
print("\nMatriz de activaciones: \n", matriz_activaciones)
print(matriz_activaciones.sum())
print("\nMatriz distancias: \n", matriz_distancias)
print("\nMedia: ", media_distancias)
print("\nNumero de grupos: ", contador_clases)


Matriz de activaciones: 
 [[ 8.  9.  2.  6.  5.  1.  0.  2.  1.  0.  3.  0.  1.  5.  0.  1.  2.  3.
   3.  1.]
 [ 1.  0.  0.  1.  4.  0.  0.  1.  3.  1.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  1.]
 [ 0.  1.  4.  2.  0.  0.  2.  0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  1.]
 [ 3.  0.  1.  0.  5.  0.  3.  0.  0.  2.  1.  0.  0.  3.  1.  1.  0.  2.
   1.  0.]
 [ 7.  0.  2.  1.  0.  0.  0.  1.  1.  0.  0.  1.  2.  0.  0.  0.  3.  0.
   0.  3.]
 [ 1.  5.  2.  3.  1.  0.  0.  0.  0.  0.  0.  1.  2.  0.  0.  0.  0.  1.
   0.  1.]
 [ 6.  0.  2.  3.  2.  0.  1.  1.  0.  5.  2.  0.  1.  3.  0.  2.  3.  0.
   0.  0.]
 [ 1.  0.  0.  1.  0.  0.  2.  0.  4.  0.  0.  0.  1.  0.  0.  1.  2. 12.
   0.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.  1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  2.
   0.  1.]
 [ 3.  2.  0.  4.  1.  1.  3.  0.  4.  2.  0.  1.  0.  2.  2.  6.  0.  0.
   0.  1.]
 [ 0.  3.  6.  0.  0.  0.  3.  0.  0.  1.  4.  0.  0.  0.  0.  0.  2.  0.
   0.  4.]
 [ 6.  2.  0.  0.  2.  3.  0.  0.  1. 